## Ingesting PDF

In [27]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
from langchain_community.document_loaders import PyPDFLoader

file_path = r"C:\Users\Joel\Dropbox\My PC (LAPTOP-RQL6K54S)\Downloads\2.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()
# Preview first page
data[0].page_content

"Artificial intelligence (AI) is probably the defining \ntechnology of the last decade, and perhaps also \nthe next. This report provides an accessible review of how it \nworks, why it matters and what we can \ndo in response to the challenges it raises. \nSince the earliest days of AI, its definition has focused on the \nability to behave with the appearance \nof intelligence. Various forms of 'Turing test' declare \nmachines as intelligent when humans cannot \ndifferentiate their actions from those of a human. Today's \ndefinitions of AI often include other \nrequirements such as autonomy, and allow intelligence to be \nlimited to specific domains. Rather \nthan contributing to the proliferation of definitions,\n1 this report adopts that of the 2018 European \nCommission Communication,\n2which is both accessible and typical of contemporary \ndefinitions:\nAI refers to systems that display intelligent behaviour by \nanalysing their environment and \ntaking action – with some degree of

In [34]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

## Vector Embeddings

In [35]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [36]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED      
nomic-embed-text:latest	0a109f422b47	274 MB	2 seconds ago	
llama3:latest          	365c0bd3c000	4.7 GB	2 days ago   	
openhermes:latest      	95477a2659b7	4.1 GB	2 days ago   	
llama2:latest          	78e26419b446	3.8 GB	9 days ago   	


In [37]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [39]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)


In [40]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 3/3 [00:11<00:00,  3.95s/it]


## Retrieval

In [41]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [42]:
# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)

In [43]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [44]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [45]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [46]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.54s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.16s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings

'Based on the provided context, AI refers to systems that display intelligent behavior by analyzing their environment and taking action – with some degree of autonomy – to achieve specific goals. This definition includes a wide range of technologies and applications that have little more in common than their apparent intelligence, which remains open to interpretation.'

In [47]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.47s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.16s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings

'Based on the provided context, this document appears to be a report about Artificial Intelligence (AI), its definition, capabilities, and implications. The document aims to provide an accessible review of AI technology, its impacts, and options for responding to the challenges it raises.'

In [48]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.32s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.16s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))